In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))

    return epochs, pp_directory

In [ ]:
# #get index of trials from behavioral data
# def epoch_index(task, listval, pp, target = np.nan):
#     if task == 'rot':
#         df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ROT_learningcurve_degrees.csv')
#     elif task == 'rdm':
#         df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMROT_learningcurve_degrees_%02d.csv' % target)
#     elif task == 'aln':
#         df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
#     elif task == 'mir':
#         df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\MIR_learningcurve_degrees_%02d.csv' % target)
   
#     ppdata = list(df.iloc[:,pp+1])
    
#     index = [i for i,v in enumerate(ppdata) if listval[0] < v < listval[1] or listval[2] < v < listval[3]]
    
#     #epochs_out = epochs[index]
#     numtrials = len(index)
    
#     #return epochs_out, numtrials
#     return index, numtrials

In [ ]:
def direction_index(task, direction, pp):
    if task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\aln_MovementWorkspace_direction.csv')
    elif task == 'rdm':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdm_MovementWorkspace_direction.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_MovementWorkspace_direction.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_MovementWorkspace_direction.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if direction == v]

    numtrials = len(index)
    
    return index, numtrials

In [ ]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]

        elif task == 'rdm':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'rdm':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [ ]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task, training):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s_block%s-epo.fif' % (pp_num, condname, task, training))
    data.save(out_fname, overwrite = True)

In [ ]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task, training):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s_block%s-ave.fif' % (pp_num, condname, task, training))
    evoked.save(out_fname, overwrite = True)

In [ ]:
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], pp)
        aldata = epoch_trials('aln', pp, data, diridx)
        output_condition_epochs(pp, aldata, ppdir, conditionnames[cond], 'aligned', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, aldata, ppdir, conditionnames[cond], 'aligned', 'all')

In [ ]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']

#aligned
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in participants:
        id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], 'aligned', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
        if condition == 0:
            aln_right_grand = mne.grand_average(evoked_list)
            aln_right_flist = evoked_list
        elif condition == 1:
            aln_left_grand = mne.grand_average(evoked_list)
            aln_left_flist = evoked_list

In [ ]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ROTATED and MIRROR TRAINING
tasks = ['rot', 'mir']

#split into training blocks
#remove first 2 trials - no feedback yet, preparing as if aligned
#22 trials per block -> 4 blocks
b0 = list(range(2,24))
b1 = list(range(24,46))
b2 = list(range(46,68))
b3 = list(range(68,90))
#b4 = list(range(64,90))
#b5 = list(range(60,72))
#b6 = list(range(72,90))
#training = [b0, b1, b2, b3, b4, b5, b6]
training = [b0, b1, b2, b3]

       
#rot and mir
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for t in range(0, len(training)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                diridx = [i for i in diridx if i in training[t]]
#                 print(diridx)
                if len(diridx) == 0:
                    break
                pdata = epoch_trials(tasks[task], pp, data, diridx)
#                 print(t, cond)
#                 print(pdata)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)

In [ ]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rot', 'mir']
#training = [0, 1, 2, 3, 4, 5, 6]
training = [0, 1, 2, 3]
#rot and mir
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for t in range(0, len(training)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], tasks[task], training[t]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
                if task == 0 and condition == 0 and t == 0:
                    rot_right_b0_grand = mne.grand_average(evoked_list)
                    rot_right_b0_flist = evoked_list
                elif task == 0 and condition == 0 and t == 1:
                    rot_right_b1_grand = mne.grand_average(evoked_list)
                    rot_right_b1_flist = evoked_list
                elif task == 0 and condition == 0 and t == 2:
                    rot_right_b2_grand = mne.grand_average(evoked_list)
                    rot_right_b2_flist = evoked_list
                elif task == 0 and condition == 0 and t == 3:
                    rot_right_b3_grand = mne.grand_average(evoked_list)
                    rot_right_b3_flist = evoked_list
                elif task == 0 and condition == 1 and t == 0:
                    rot_left_b0_grand = mne.grand_average(evoked_list)
                    rot_left_b0_flist = evoked_list
                elif task == 0 and condition == 1 and t == 1:
                    rot_left_b1_grand = mne.grand_average(evoked_list)
                    rot_left_b1_flist = evoked_list
                elif task == 0 and condition == 1 and t == 2:
                    rot_left_b2_grand = mne.grand_average(evoked_list)
                    rot_left_b2_flist = evoked_list
                elif task == 0 and condition == 1 and t == 3:
                    rot_left_b3_grand = mne.grand_average(evoked_list)
                    rot_left_b3_flist = evoked_list
                elif task == 1 and condition == 0 and t == 0:
                    mir_right_b0_grand = mne.grand_average(evoked_list)
                    mir_right_b0_flist = evoked_list
                elif task == 1 and condition == 0 and t == 1:
                    mir_right_b1_grand = mne.grand_average(evoked_list)
                    mir_right_b1_flist = evoked_list
                elif task == 1 and condition == 0 and t == 2:
                    mir_right_b2_grand = mne.grand_average(evoked_list)
                    mir_right_b2_flist = evoked_list
                elif task == 1 and condition == 0 and t == 3:
                    mir_right_b3_grand = mne.grand_average(evoked_list)
                    mir_right_b3_flist = evoked_list
                elif task == 1 and condition == 1 and t == 0:
                    mir_left_b0_grand = mne.grand_average(evoked_list)
                    mir_left_b0_flist = evoked_list
                elif task == 1 and condition == 1 and t == 1:
                    mir_left_b1_grand = mne.grand_average(evoked_list)
                    mir_left_b1_flist = evoked_list
                elif task == 1 and condition == 1 and t == 2:
                    mir_left_b2_grand = mne.grand_average(evoked_list)
                    mir_left_b2_flist = evoked_list
                elif task == 1 and condition == 1 and t == 3:
                    mir_left_b3_grand = mne.grand_average(evoked_list)
                    mir_left_b3_flist = evoked_list

In [ ]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#RANDOM
tasks = ['rdm']

#split into training blocks
b0 = list(range(2,48))
#b1 = list(range(16,32))
#b2 = list(range(32,48))
#b3 = list(range(36,48))

#training = [b0, b1, b2, b3]
training = [b0]

       
#rdm
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for t in range(0, len(training)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                diridx = [i for i in diridx if i in training[t]]
#                 print(diridx)
                if len(diridx) == 0:
                    break
                pdata = epoch_trials(tasks[task], pp, data, diridx)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)

In [ ]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rdm']
#training = [0, 1, 2, 3]
training = [0]
#rdm
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for t in range(0, len(training)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], tasks[task], training[t]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
                if task == 0 and condition == 0 and t == 0:
                    rdm_right_b0_grand = mne.grand_average(evoked_list)
                    rdm_right_b0_flist = evoked_list
                elif task == 0 and condition == 1 and t == 0:
                    rdm_left_b0_grand = mne.grand_average(evoked_list)
                    rdm_left_b0_flist = evoked_list

In [ ]:
#runs for LRP ONLY: need separate files for C3 and C4

# flists = ['aln_right', 'aln_left',
#           'rot_right_b0', 'rot_right_b1', 'rot_right_b2', 'rot_right_b3', 'rot_right_b4', 'rot_right_b5', 'rot_right_b6',
#           'rot_left_b0', 'rot_left_b1', 'rot_left_b2', 'rot_left_b3', 'rot_left_b4', 'rot_left_b5', 'rot_left_b6',
#           'mir_right_b0', 'mir_right_b1', 'mir_right_b2', 'mir_right_b3', 'mir_right_b4', 'mir_right_b5', 'mir_right_b6',
#           'mir_left_b0', 'mir_left_b1', 'mir_left_b2', 'mir_left_b3', 'mir_left_b4', 'mir_left_b5', 'mir_left_b6',
#           'rdm_right_b0', 'rdm_right_b1', 'rdm_right_b2', 'rdm_right_b3',
#           'rdm_left_b0', 'rdm_left_b1', 'rdm_left_b2', 'rdm_left_b3']

# flists = ['aln_right', 'aln_left',
#           'rot_right_b0', 'rot_right_b1', 'rot_right_b2', 'rot_right_b3', 'rot_right_b4',
#           'rot_left_b0', 'rot_left_b1', 'rot_left_b2', 'rot_left_b3', 'rot_left_b4', 
#           'mir_right_b0', 'mir_right_b1', 'mir_right_b2', 'mir_right_b3', 'mir_right_b4',
#           'mir_left_b0', 'mir_left_b1', 'mir_left_b2', 'mir_left_b3', 'mir_left_b4',
#           'rdm_right_b0', 'rdm_right_b1', 'rdm_right_b2',
#           'rdm_left_b0', 'rdm_left_b1', 'rdm_left_b2']

flists = ['aln_right', 'aln_left',
          'rot_right_b0', 'rot_right_b1', 'rot_right_b2', 'rot_right_b3',
          'rot_left_b0', 'rot_left_b1', 'rot_left_b2', 'rot_left_b3',
          'mir_right_b0', 'mir_right_b1', 'mir_right_b2', 'mir_right_b3',
          'mir_left_b0', 'mir_left_b1', 'mir_left_b2', 'mir_left_b3',
          'rdm_right_b0', 'rdm_left_b0']

channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        if f == 'aln_right':
            for pp in range(0, len(aln_right_flist)):
                df = aln_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'aln_left':
            for pp in range(0, len(aln_left_flist)):
                df = aln_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b0':
            for pp in range(0, len(rot_right_b0_flist)):
                df = rot_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b1':
            for pp in range(0, len(rot_right_b1_flist)):
                df = rot_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b2':
            for pp in range(0, len(rot_right_b2_flist)):
                df = rot_right_b2_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b3':
            for pp in range(0, len(rot_right_b3_flist)):
                df = rot_right_b3_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
#         elif f == 'rot_right_b4':
#             for pp in range(0, len(rot_right_b4_flist)):
#                 df = rot_right_b4_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rot_right_b5':
#             for pp in range(0, len(rot_right_b5_flist)):
#                 df = rot_right_b5_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rot_right_b6':
#             for pp in range(0, len(rot_right_b6_flist)):
#                 df = rot_right_b6_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b0':
            for pp in range(0, len(rot_left_b0_flist)):
                df = rot_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b1':
            for pp in range(0, len(rot_left_b1_flist)):
                df = rot_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b2':
            for pp in range(0, len(rot_left_b2_flist)):
                df = rot_left_b2_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b3':
            for pp in range(0, len(rot_left_b3_flist)):
                df = rot_left_b3_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
#         elif f == 'rot_left_b4':
#             for pp in range(0, len(rot_left_b4_flist)):
#                 df = rot_left_b4_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rot_left_b5':
#             for pp in range(0, len(rot_left_b5_flist)):
#                 df = rot_left_b5_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rot_left_b6':
#             for pp in range(0, len(rot_left_b6_flist)):
#                 df = rot_left_b6_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b0':
            for pp in range(0, len(mir_right_b0_flist)):
                df = mir_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b1':
            for pp in range(0, len(mir_right_b1_flist)):
                df = mir_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b2':
            for pp in range(0, len(mir_right_b2_flist)):
                df = mir_right_b2_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b3':
            for pp in range(0, len(mir_right_b3_flist)):
                df = mir_right_b3_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
#         elif f == 'mir_right_b4':
#             for pp in range(0, len(mir_right_b4_flist)):
#                 df = mir_right_b4_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'mir_right_b5':
#             for pp in range(0, len(mir_right_b5_flist)):
#                 df = mir_right_b5_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'mir_right_b6':
#             for pp in range(0, len(mir_right_b6_flist)):
#                 df = mir_right_b6_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b0':
            for pp in range(0, len(mir_left_b0_flist)):
                df = mir_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b1':
            for pp in range(0, len(mir_left_b1_flist)):
                df = mir_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b2':
            for pp in range(0, len(mir_left_b2_flist)):
                df = mir_left_b2_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b3':
            for pp in range(0, len(mir_left_b3_flist)):
                df = mir_left_b3_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
#         elif f == 'mir_left_b4':
#             for pp in range(0, len(mir_left_b4_flist)):
#                 df = mir_left_b4_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'mir_left_b5':
#             for pp in range(0, len(mir_left_b5_flist)):
#                 df = mir_left_b5_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'mir_left_b6':
#             for pp in range(0, len(mir_left_b6_flist)):
#                 df = mir_left_b6_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_b0':
            for pp in range(0, len(rdm_right_b0_flist)):
                df = rdm_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
#         elif f == 'rdm_right_b1':
#             for pp in range(0, len(rdm_right_b1_flist)):
#                 df = rdm_right_b1_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rdm_right_b2':
#             for pp in range(0, len(rdm_right_b2_flist)):
#                 df = rdm_right_b2_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rdm_right_b3':
#             for pp in range(0, len(rdm_right_b3_flist)):
#                 df = rdm_right_b3_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_b0':
            for pp in range(0, len(rdm_left_b0_flist)):
                df = rdm_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,300):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
#         elif f == 'rdm_left_b1':
#             for pp in range(0, len(rdm_left_b1_flist)):
#                 df = rdm_left_b1_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rdm_left_b2':
#             for pp in range(0, len(rdm_left_b2_flist)):
#                 df = rdm_left_b2_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
#         elif f == 'rdm_left_b3':
#             for pp in range(0, len(rdm_left_b3_flist)):
#                 df = rdm_left_b3_flist[pp].to_data_frame()
#                 time = df['time'].tolist()
#                 df = df[[channel]]
#                 ppdat = []
#                 for timept in range(0,300):
#                     subdf = df.loc[timept,:].mean()
#                     ppdat.append(subdf)
#                 newdf['pp%03d'% pp] = ppdat
#             newdf['time'] = time
#             newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, channel))
#             newdf.to_csv(newdf_filename)
             


In [ ]:
# # Difference waves between aligned and perturbed conditions for C3
# #channels = ['C3']
# channels = ['C4']
# # channels = ['Oz','O1','O2']
# evokeds = dict(aln_right = right_aln_flist, 
#                aln_left = left_aln_flist,
#                rot_right = right_rot_flist,
#                rot_left = left_rot_flist,
#                rdm_right = right_rdm_flist,
#                rdm_left = left_rdm_flist,
#                mir_right = right_mir_flist,
#                mir_left = left_mir_flist)
# #plot
# mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)